## 텍스트 유사도 Text Similarity
문장간의 의미 유사도를 계산하는 일은 매우 중요<br>
임베딩을 통해 단어들의 벡터를 구한 다음 벡터간의 거리를 계산하는 방법으로 유사도를 구함<br>

> 챗봇 엔진에 입력되는 문장과 시스템에서 해당 주제의 답변과 연관되어 있는 질문이 얼마나 유사한지 계산할 수 있어야 적절한 답변 출력 가능 <br>
Word2Vec은 인공 신경망을 이용했고 이번에는 통계적인 방법을 사용할 것 <br>
상황에 따라 방법에 따라 책봇 엔진 성능 향상이 달라진다.

### n-gram 유사도
주어진 문장 n 개의 연속적인 단어 시권스를 의미<br>
n-gram은 문장에서 n개의 단어를 토큰으로 사용<br>
n-gram으로 비교하면 단어의 출현 빈도에 기반한 유사도 계산 가능
- 인용, 도용 조사할 때 사용

___


n = 1: 유니그램 unigram<br>
n = 2: 바이그램 bigram<br>
n = 3: 트라이그램 trigram<br>
n >= 4: n-그램 n-gram

___

similarity = tf(A, B)/tokens(A)

tf: A, B에서 동일한 토큰의 출현 빈도<br>
tokens: 해당 문장에서 전체 토큰 수

In [5]:
from konlpy.tag import Komoran

# 어절 단위로 n-gram 토큰 추출
def word_ngram(bow, num_gram):
    text = tuple(bow)
    ngrams = [text[x:x + num_gram] for x in range(0, len(text))]
    return tuple(ngrams)

# 유사도 계산
def similarity(doc1, doc2):
    cnt = 0
    for token in doc1:
        if token in doc2:
            cnt = cnt + 1
    return cnt/len(doc1)

# 문장 정의
sentence1 = '3월에 니체는 친구의 제안으로 트리니티에 입학했다.'
sentence2 = '3월에 니체는 친구의 제안으로 대학교에 입학했다.'
sentence3 = '나는 맛있는 밥을 친구와 함께 먹었다.'

# 형태소 분석기에서 명사(단어) 추출
komoran = Komoran()
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

# 단어 n-gram 토큰 추출
doc1 = word_ngram(bow1, 2)
doc2 = word_ngram(bow2, 2)
doc3 = word_ngram(bow3, 2)

# 추출된 n-gram 토큰 출력
print(doc1)
print(doc2)

# 유사도 계산
r1 = similarity(doc1, doc2)
r2 = similarity(doc3, doc1)

# 계산된 유사도 출력
print(r1)
print(r2)

(('3월', '니체'), ('니체', '친구'), ('친구', '제안'), ('제안', '트리니티'), ('트리니티', '입학'), ('입학',))
(('3월', '니체'), ('니체', '친구'), ('친구', '제안'), ('제안', '대학교'), ('대학교', '입학'), ('입학',))
0.6666666666666666
0.0


n-gram은 모든 단어의 출현 빈도를 확인하는 것이 아닌 연속되는 문장에서 일부 단어(n만큼) 확인하여,<br>
전체 문장을 고려한 언어 모델보다 정확도가 떨어질 수 있다.<br>

또한 n을 크게 잡을 경우 비교 문장의 토큰솨 비교할 때 카운트롤 놓치 확률도 커지고, <br>
n을 작게 잡을 경우 카운트 확률은 높아지지만 문맥을 파악하는 정확도는 낮아지는 구조이다.<br>

> n-gram 모델에서 n의 설정은 매우 중요하다.

### 코사인 유사도
두 벡터 간 코사인 각도를 이용해 유사도를 측정하는 방법 <br>
벡터의 크기가 중요하지 않을 때 거리를 측정하기 위하여 사용<br>
출현 빈도로 계산시 동일한 단
어가 많이 포함되어 있을수록 벡터의 크기가 커지고, 코사인 유사도는 크기와 상관없이 안정적이다.<br>


In [6]:
from konlpy.tag import Komoran
import numpy as np
from numpy import dot # numpy array를 곱할 때 사용
from numpy.linalg import norm

# 코사인 유사도 계산
def cos_sim(vec1, vec2):
    return(vec1, vec2) / (norm(vec1) * norm(vec2))

# TDM(단어 문서 행렬) 만들기
def make_term_doc_mat(sentence_bow, word_dics):
    freq_mat = {}
    
    for word in word_dics:
        freq_mat[word] = 0
        
    for word in word_dics:
        if word in sentence_bow:
            freq_mat[word] += 1
            
    return freq_mat

# 단어 벡터 만들기
def make_vector(tdm):
    vec = []
    for key in tdm:
        vec.append(tdm[key])
    return vec

# 문장 정의
sentence1 = '3월에 니체는 친구의 제안으로 트리니티에 입학했다.'
sentence2 = '3월에 니체는 친구의 제안으로 대학교에 입학했다.'
sentence3 = '나는 맛있는 밥을 친구와 함께 먹었다.'

# 형태소 분석기를 이용해 단어 묶음 리스트 생성
komoran = Komoran()
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

# 단어 묶음 리스트를 하나로 합침
bow = bow1 + bow2 + bow3

# 단어 묶음에서 중복을 제거해 단어 사전 구축
word_dics = []
for token in bow:
    if token not in word_dics:
        word_dics.append(token)
        
# 문장별 단어 문서 행렬 계산
freq_list1 = make_term_doc_mat(bow1, word_dics)
freq_list2 = make_term_doc_mat(bow2, word_dics)
freq_list3 = make_term_doc_mat(bow3, word_dics)
print(freq_list1)
print(freq_list2)
print(freq_list3)

# 문장 벡터 생성
doc1 = np.array(make_vector(freq_list1))
doc2 = np.array(make_vector(freq_list2))
doc3 = np.array(make_vector(freq_list3))

# 코사인 유사도 계산
r1 = cos_sim(doc1, doc2)
r2 = cos_sim(doc3, doc1)
print(r1)
print(r2)

{'3월': 1, '니체': 1, '친구': 1, '제안': 1, '트리니티': 1, '입학': 1, '대학교': 0, '밥': 0}
{'3월': 1, '니체': 1, '친구': 1, '제안': 1, '트리니티': 0, '입학': 1, '대학교': 1, '밥': 0}
{'3월': 0, '니체': 0, '친구': 1, '제안': 0, '트리니티': 0, '입학': 0, '대학교': 0, '밥': 1}
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667
  0.         0.        ]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.         0.16666667
  0.16666667 0.        ]]
[[0.         0.         0.28867513 0.         0.         0.
  0.         0.28867513]
 [0.28867513 0.28867513 0.28867513 0.28867513 0.28867513 0.28867513
  0.         0.        ]]


자연어 처리 과정
1. 토크나이징
2. 임베딩
3. 유사도 계산